In [41]:
import base64
import os
from cryptography.hazmat.primitives.ciphers.aead import AESGCM

# Tạo key bí mật (16 bytes = 128-bit)
# Nếu muốn giữ cố định key để giải mã, lưu key này ra file hoặc DB
key = AESGCM.generate_key(bit_length=128)
aesgcm = AESGCM(key)

def encrypt_urlsafe(plaintext: str) -> str:
    nonce = os.urandom(12)  # 96-bit IV
    salt = os.urandom(8)    # Thêm salt để khác biệt mỗi lần
    data = salt + plaintext.encode()
    ciphertext = aesgcm.encrypt(nonce, data, None)
    token_bytes = nonce + ciphertext
    return base64.urlsafe_b64encode(token_bytes).decode()

def decrypt_urlsafe(token: str) -> str:
    token_bytes = base64.urlsafe_b64decode(token.encode())
    nonce = token_bytes[:12]
    ciphertext = token_bytes[12:]
    data = aesgcm.decrypt(nonce, ciphertext, None)
    salt = data[:8]
    return data[8:].decode()

# Ví dụ chạy
print("Key (base64):", base64.urlsafe_b64encode(key).decode())

for _ in range(4):
    cipher = encrypt_urlsafe("page=home")
    print("Cipher:", cipher, "| Giải mã lại:", decrypt_urlsafe(cipher))


Key (base64): U3qsr_mcAz6AuYrmjeWKOg==
Cipher: NYGeAk7uIogWoYYvYWpluUBTFjTzBGY9MPUP4UoDuKjEOuXjB8Y4NY4T90eG | Giải mã lại: page=home
Cipher: 1auP-YiKovgyxKZE7EAM77dfPdoMqq1yaQH85r3RGHtFGtZCGzV7ijS07NnW | Giải mã lại: page=home
Cipher: KNTmRItZvWUS21dUeu-fo0VP_CVl71Mi8Ky_bpndf8ofTGpP78eZ8jn7r0t7 | Giải mã lại: page=home
Cipher: qUCA_K5S2M-rPfRyJrSruWXDc__X2x-aEYDgUYm5momiKHjLrVYsqkVYmkHC | Giải mã lại: page=home
